In [113]:
import pandas as pd
import pickle
from outbreak_data import authenticate_user, outbreak_data

In [7]:
authenticate_user.authenticate_new_user()

Please open this url in a web browswer and authenticate with your GISAID credentials:  https://gpsapi.epicov.org/epi3/gps_authenticate/SMDMFCBEFONNZLFLQMRKTVHVGEPEDGDABVXYXXZNZUNKTNJCIOBQNJTSZOQKEWTDKJIJCZDLSXDPQGWDVUCDYTBUDFCCSKWJXWAEEWBAKYSBRESZMLCOUCWAWIBKOKEH
Waiting for authorization response... [Press Ctrl-C to abort]
Authenication failed, trying again in 5 seconds...
Waiting for authorization response... [Press Ctrl-C to abort]
Authenticated successfully!

    TERMS OF USE for Python Package and
    Reminder of GISAID's Database Access Agreement
    Your ability to access and use Data in GISAID, including your access and
    use of same via R Package, is subject to the terms and conditions of
    GISAID's Database Access Agreement (“DAA”) (which you agreed to
    when you requested access credentials to GISAID), as well as the
    following terms:
    1. You will treat all data contained in the R Package consistent with
    other Data in GISAID and in accordance with GISAID's Da

In [134]:
geo_isos = input("Enter the 3 capitalized letter ISO codes (e.x. BOL, BRA, CHL) you wish to examine, seperated by commas. ")
geo_isos = geo_isos.replace(" ", "").split(',')
# sars metadata 1: cases increases
cases_numIncrease = outbreak_data.cases_by_location(geo_isos, pull_smoothed=True)
print('ISOS accessed cases successfully, with shape: ', cases_numIncrease.shape)
# sars metadata 2: lineages prevalences
lineage_prevalences = []
for loc in geo_isos:
    locDf = outbreak_data.prevalence_by_location(loc, other_threshold=0.85)
    locDf['location'] = [loc] * len(locDf)
    lineage_prevalences.append(locDf)
lineage_prevalences = pd.concat(lineage_prevalences)
print('Lineage prevalences accessed successfully, with shape: ', lineage_prevalences.shape)

Enter the 3 capitalized letter ISO codes (e.x. BOL, BRA, CHL) you wish to examine, seperated by commas.  BOL, BRA, CHL


ISOS accessed cases successfully, with shape:  (3363, 5)
Lineage prevalences accessed successfully, with shape:  (20951, 7)


In [122]:
#sars metadata 3: lineage mutations
# select good candidates for mutation information
interesting_lineages = input("Enter the exact name of the lineage(s) (e.x. xbb.1.5.76, xbb.1.5) you wish to examine, seperated by commas. ")
interesting_lineages = interesting_lineages.replace(" ", "").split(',')
lineage_mutations = outbreak_data.lineage_mutations(pango_lin=interesting_lineages)
print('Lineage names used to access lineage mutations successfully, with shape: ', lineage_mutations.shape)
print('All necessary data gathered from the Outbreak API! Now moving on to preprocessing...')

Enter the exact name of the lineage(s) (e.x. xbb.1.5.76, xbb.1.5) you wish to examine, seperated by commas.  xbb.1.5.76, xbb.1.5


Lineage names used to access lineage mutations successfully, with shape:  (132, 12)
All necessary data gathered from the Outbreak API! Now moving on to preprocessing...


In [123]:
lineage_mutations

,mutation,mutation_count,lineage_count,lineage,gene,ref_aa,alt_aa,codon_num,codon_end,type,prevalence,change_length_nt
0,orf6:d61l,349,351,xbb.1.5.76,ORF6,D,L,61,None,substitution,0.994302,None
1,s:l24s,344,351,xbb.1.5.76,S,L,S,24,None,substitution,0.980057,None
2,n:r203k,350,351,xbb.1.5.76,N,R,K,203,None,substitution,0.997151,None
3,s:g339h,344,351,xbb.1.5.76,S,G,H,339,None,substitution,0.980057,None
4,s:f486p,335,351,xbb.1.5.76,S,F,P,486,None,substitution,0.954416,None
...,...,...,...,...,...,...,...,...,...,...,...,...
63,s:g446s,177573,189905,xbb.1.5,S,G,S,446,None,substitution,0.935062,None
64,s:del144/144,175806,189905,xbb.1.5,S,S:DEL144/144,DEL144/144,144,144.0,deletion,0.925758,3.0
65,s:h146q,173819,189905,xbb.1.5,S,H,Q,146,None,substitution,0.915294,None
66,s:r408s,172593,189905,xbb.1.5,S,R,S,408,None,substitution,0.908839,None


In [124]:
# Processing the Data
# Selecting most recent 5000 datapoints to plot visual of lineage, as there is too  many to plot
plot_lineage_prevalences=lineage_prevalences.sort_values('date')[-5000:]

In [125]:
plot_lineage_prevalences.head()

,date,total_count,lineage_count,lineage,prevalence,prevalence_rolling,location
4420,2022-04-10,0,0,other,0.000000,0.000000,BOL
3056,2022-04-10,0,0,ba.5.1,0.000000,0.000000,BRA
6482,2022-04-10,0,0,p.1,0.000000,0.000000,BRA
4129,2022-04-11,73,26,ba.1.1,0.356164,0.401914,CHL
3498,2022-04-11,0,0,ba.5.2.1,0.000000,0.000000,BRA


In [126]:
# Calculates the cumulative sum of lineages across time, across all locations - in the past 5000 recent datapoints
plot_lineage_prevalences['prevalence_cumSum'] = plot_lineage_prevalences.groupby(['date', 'lineage'])['prevalence_rolling'].cumsum()
most_recent_lineages = plot_lineage_prevalences.loc[:,['prevalence_cumSum', 'lineage']].groupby('lineage').mean().sort_values('prevalence_cumSum')[-30:].index.to_list()

In [127]:
plot_lineage_prevalences.head()

,date,total_count,lineage_count,lineage,prevalence,prevalence_rolling,location,prevalence_cumSum
4420,2022-04-10,0,0,other,0.000000,0.000000,BOL,0.000000
3056,2022-04-10,0,0,ba.5.1,0.000000,0.000000,BRA,0.000000
6482,2022-04-10,0,0,p.1,0.000000,0.000000,BRA,0.000000
4129,2022-04-11,73,26,ba.1.1,0.356164,0.401914,CHL,0.401914
3498,2022-04-11,0,0,ba.5.2.1,0.000000,0.000000,BRA,0.000000


In [128]:
# Selecting top 20 most prevalent in the most recent datapoints to limit visual complexity
plot_recent_lineage_prevalences = plot_lineage_prevalences.where(plot_lineage_prevalences.lineage.apply(lambda x: x in most_recent_lineages)).dropna(how='all')
plot_recent_lineage_prevalences = plot_recent_lineage_prevalences.reset_index(drop=True)
#splitting the dataset into lineage_defined and lineage_undefined
lineage_undefined = plot_recent_lineage_prevalences.where(plot_recent_lineage_prevalences.lineage == 'other').dropna(how='all')
lineage_defined = plot_recent_lineage_prevalences.where(plot_recent_lineage_prevalences.lineage != 'other').dropna(how='all')

In [129]:
lineage_defined.shape

(3658, 8)

In [130]:
lineage_undefined.shape

(1342, 8)

In [131]:
# For the lineage_defined dataset, we can go ahead and merge it with the mutations data
sars_epi_viro = pd.merge(lineage_defined, lineage_mutations, on = 'lineage', suffixes=('_cases','_mutations'))

In [132]:
print('Log 1: ', sars_epi_viro.shape)

Log 1:  (27820, 19)


In [135]:
# The _score, admin1 columns are not meaningful and could be dropped. (it is used by the API team)
cases_numIncrease = cases_numIncrease.drop('_score', axis=1)
cases_numIncrease = cases_numIncrease.drop('admin1', axis=1)
# _id column contains the location name, and this is the only unique piece of info 
# feature can be re-engineered to location & then merged with the sars_epi_cov dataset
cases_numIncrease['location'] = cases_numIncrease._id.apply(lambda x: x[:3])
cases_numIncrease = cases_numIncrease.drop('_id', axis = 1)
# Finally, we constructed the training/test dataset for a M.L/A.I model
# It should contain all necessary features for this project
sars_epi_viro = pd.merge(sars_epi_viro, cases_numIncrease, on=['location', 'date'])
print("Full corpus created, with shape: ", sars_epi_viro.shape)
#Encoding the categorical variables in the data for the model
categs = ['date', 'lineage', 'location', 'mutation', 'gene', 'ref_aa', 'alt_aa', 'codon_end', 'type', 'change_length_nt']
for col in categs:
    sars_epi_viro[col] = sars_epi_viro[col].astype('category')
sars_epi_viro_encoded = pd.get_dummies(sars_epi_viro, columns=categs, drop_first=True)

Full corpus created, with shape:  (9448, 20)


In [136]:
sars_epi_viro_encoded.shape

(9448, 226)

In [138]:
sars_epi_viro_encoded.columns.to_list()

['total_count',
 'lineage_count_cases',
 'prevalence_cases',
 'prevalence_rolling',
 'prevalence_cumSum',
 'mutation_count',
 'lineage_count_mutations',
 'codon_num',
 'prevalence_mutations',
 'confirmed_rolling',
 'date_2022-12-10',
 'date_2022-12-11',
 'date_2022-12-12',
 'date_2022-12-13',
 'date_2022-12-14',
 'date_2022-12-15',
 'date_2022-12-16',
 'date_2022-12-17',
 'date_2022-12-18',
 'date_2022-12-19',
 'date_2022-12-20',
 'date_2022-12-21',
 'date_2022-12-22',
 'date_2022-12-23',
 'date_2022-12-24',
 'date_2022-12-25',
 'date_2022-12-26',
 'date_2022-12-27',
 'date_2022-12-28',
 'date_2022-12-29',
 'date_2022-12-30',
 'date_2022-12-31',
 'date_2023-01-01',
 'date_2023-01-02',
 'date_2023-01-03',
 'date_2023-01-04',
 'date_2023-01-05',
 'date_2023-01-06',
 'date_2023-01-07',
 'date_2023-01-08',
 'date_2023-01-09',
 'date_2023-01-10',
 'date_2023-01-11',
 'date_2023-01-12',
 'date_2023-01-13',
 'date_2023-01-14',
 'date_2023-01-15',
 'date_2023-01-16',
 'date_2023-01-17',
 'date

### Developing the model

In [144]:
with open('sars_lasso_model.pkl', 'rb') as file:
    lasso_model = pickle.load(file)

In [146]:
if hasattr(lasso_model, 'feature_names_in_'):
    lasso_model.feature_names_in_ = None

In [147]:
predictions = lasso_model.predict(sars_epi_viro_encoded.drop('prevalence_rolling', axis=1))

/opt/conda/envs/test-env/lib/python3.9/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


In [148]:
predictions = pd.Series(predictions)

In [150]:
predictions.to_csv('testpred.csv', index=False)